In [1]:
import nltk
import keras
import numpy as np
import sklearn

from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder

from keras import backend as K
from keras import layers
from keras import models
from keras.models import load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

import tensorflow as tf


Using TensorFlow backend.


In [2]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    words = sentence[index];
    return {
    'words': words,
    'is_first': index == 0,
    'is_last': index == len(sentence) - 1,
    'is_capitalized': words[0].upper() == words[0],
    'is_all_caps': words.upper() == words,
    'is_all_lower': words.lower() == words,
    'prefix-1': words[0],
    'prefix-2': words[:2],
    'prefix-3': words[:3],
    'prefix-4': words[:4],
    'suffix-1': words[-1],
    'suffix-2': words[-2:],
    'suffix-3': words[-3:],
    'prev_word': '' if index == 0 else sentence[index - 1],
    'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
    'is_numeric': words.isdigit(),

}

In [3]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()

train, test = sklearn.model_selection.train_test_split(tagged_sentences, test_size=0.3, random_state=1);




In [4]:
def untag(tagged_sentence):
    return[w for w, t in tagged_sentence];
def transform(sentence):
    
    X, y = [],[];
    for x in sentence[:1000]:

        for i in range(len(x)):
            
            X.append(features(untag(x),i));
            y.append(x[i][1]);


    return X, y;

In [5]:
X_train, y_train = transform(train);
X_test, y_test = transform(test);

vectorizer = DictVectorizer(sparse=False);
vectorizer.fit(X_train + X_test);



X_test = vectorizer.transform(X_test)
X_train = vectorizer.transform(X_train)



encoder = LabelEncoder();
encoder.fit(y_train + y_test);

y_train = encoder.transform(y_train);
y_test = encoder.transform(y_test);


y_train =np_utils.to_categorical(y_train);
y_test = np_utils.to_categorical(y_test);


In [6]:
def model(input_dim, hidden_neurons, output_dim):
    model = models.Sequential([
        Dense(hidden_neurons, input_dim=input_dim, activation='relu'),
        Dropout(0.3),
        Dense(hidden_neurons, activation='relu'),
        Dropout(0.3),
        Dense(output_dim, activation='sigmoid')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

mod = model(X_train.shape[1], 128, y_train.shape[1])


mod.fit(X_train, y_train, epochs=5, validation_split =0.1, batch_size = 32)


Train on 22869 samples, validate on 2541 samples
Epoch 1/5
22869/22869 [==============================] - 133s 6ms/step - loss: 0.9029 - acc: 0.7536 - val_loss: 0.2497 - val_acc: 0.9209
Epoch 2/5
22869/22869 [==============================] - 123s 5ms/step - loss: 0.1903 - acc: 0.9441 - val_loss: 0.1938 - val_acc: 0.9362
Epoch 3/5
22869/22869 [==============================] - 121s 5ms/step - loss: 0.0969 - acc: 0.9728 - val_loss: 0.1944 - val_acc: 0.9441
Epoch 4/5
22869/22869 [==============================] - 123s 5ms/step - loss: 0.0647 - acc: 0.9805 - val_loss: 0.2046 - val_acc: 0.9433
Epoch 5/5
22869/22869 [==============================] - 122s 5ms/step - loss: 0.0475 - acc: 0.9852 - val_loss: 0.2024 - val_acc: 0.9481


In [7]:
#mod.save('NLP_model.h5')

m = load_model('NLP_model.h5')

In [8]:

score = m.evaluate(X_test, y_test)
print("The score is: %f",score[1])

25675/25675 [==============================] - 67s 3ms/step
The score is: %f 0.9400584225877466


In [9]:
def translate(sent):
    l =[]
    index = 0
    for i in sent:
        l.append(features(sent,index))
        index += 1

    v = vectorizer.transform(l)
    print(v.shape)

    encoded_tags = m.predict_classes(v)

    return encoder.inverse_transform(encoded_tags)
 



In [15]:
translate(["I","am", "working" ])


(5, 36528)


array(['PRP', 'VBP', 'JJ', 'TO', 'VB'], dtype='<U6')